# Clustering : Segmentation de produits

> Ce notebook est un exemple pratique clef en main pour apprendre les techniques de clustering (apprentissage non supervise) en Machine Learning.
> Nous allons segmenter un catalogue de produits en groupes homogenes a partir de leurs caracteristiques (prix, ventes, notes, etc.).
>
> **Objectifs :**
> - Comprendre les algorithmes de clustering (K-Means, DBSCAN)
> - Determiner le nombre optimal de clusters (methode du coude, score silhouette)
> - Visualiser les clusters en 2D avec une reduction de dimension (PCA)
> - Profiler les clusters pour en tirer des insights business

In [ ]:
# ============================================================
# Cellule 1 : Imports et chargement des donnees
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, silhouette_samples

import warnings
warnings.filterwarnings("ignore")

# Configuration graphique
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["figure.dpi"] = 100
sns.set_style("whitegrid")

# Chargement des donnees
df = pd.read_csv("../data/produits_clustering.csv")

print(f"Dataset charge avec succes : {df.shape[0]} lignes, {df.shape[1]} colonnes")
df.head(10)

In [ ]:
# ============================================================
# Cellule 2 : Analyse exploratoire (EDA)
# ============================================================

print("=" * 60)
print("INFORMATIONS GENERALES")
print("=" * 60)
print(f"\nDimensions : {df.shape}")
print(f"\nTypes des colonnes :")
print(df.dtypes)

print("\n" + "=" * 60)
print("VALEURS MANQUANTES")
print("=" * 60)
print(df.isnull().sum())

print("\n" + "=" * 60)
print("STATISTIQUES DESCRIPTIVES")
print("=" * 60)
print(df.describe().round(2))

# Repartition par categorie
print("\n" + "=" * 60)
print("REPARTITION PAR CATEGORIE")
print("=" * 60)
print(df["categorie"].value_counts())

# Distributions des variables numeriques
variables_num = ["prix", "nb_ventes_mensuel", "note_moyenne", "nb_avis", "stock"]

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, col in enumerate(variables_num):
    axes[i].hist(df[col], bins=15, edgecolor="black", alpha=0.7, color="steelblue")
    axes[i].set_title(f"Distribution de {col}")
    axes[i].set_xlabel(col)
    axes[i].set_ylabel("Frequence")

# Masquer le dernier subplot inutilise
axes[5].set_visible(False)

plt.tight_layout()
plt.show()

# Boxplot par categorie
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.boxplot(data=df, x="categorie", y="prix", ax=axes[0], palette="Set2")
axes[0].set_title("Prix par categorie")
axes[0].tick_params(axis="x", rotation=45)

sns.boxplot(data=df, x="categorie", y="nb_ventes_mensuel", ax=axes[1], palette="Set2")
axes[1].set_title("Ventes mensuelles par categorie")
axes[1].tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

## Preprocessing des donnees

### Pourquoi preparer les donnees ?

Le clustering regroupe les produits "qui se ressemblent". Mais comment mesurer la ressemblance ?

L'algorithme calcule la **distance** entre les produits. Le probleme : si le prix va de 5€ a 900€ et les notes de 1 a 5, le prix va completement **ecraser** l'influence des notes (parce que les ecarts sont beaucoup plus grands).

> **Analogie** : C'est comme comparer des temperatures en Celsius et en Fahrenheit. 30°C et 86°F, c'est pareil ! Mais si on met les deux dans un calcul sans convertir, on obtient n'importe quoi.

La **standardisation** remet toutes les variables sur la meme echelle (moyenne = 0, ecart-type = 1). Ainsi, chaque variable pese le meme poids dans le calcul de distance.

### Etapes de preparation :
1. **Selection** des variables numeriques pertinentes
2. **Encodage** de la variable categorielle (transformer "Electronique" en nombre)
3. **Standardisation** : centrer et reduire chaque variable

In [ ]:
# ============================================================
# Cellule 3 : Preprocessing (encodage + scaling)
# ============================================================

# Selection des features pour le clustering
# On utilise les variables numeriques + la categorie encodee
features_clustering = ["prix", "nb_ventes_mensuel", "note_moyenne", "nb_avis", "stock"]

# Encodage de la categorie
le = LabelEncoder()
df["categorie_encoded"] = le.fit_transform(df["categorie"])

# Ajout de la categorie encodee aux features
features_avec_cat = features_clustering + ["categorie_encoded"]

# Preparation de la matrice de features
X = df[features_avec_cat].copy()

print("Features utilisees pour le clustering :")
print(X.columns.tolist())
print(f"\nDimensions : {X.shape}")

# Standardisation (centrage-reduction)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\n--- Donnees apres standardisation ---")
X_scaled_df = pd.DataFrame(X_scaled, columns=features_avec_cat)
print(X_scaled_df.describe().round(2))
print("\nVerification : moyennes proches de 0 et ecarts-types proches de 1.")

## Determination du nombre optimal de clusters

Le premier defi du clustering : **combien de groupes** faut-il creer ? Trop peu = on melange des produits tres differents. Trop = chaque produit est presque seul dans son groupe.

### Methode 1 : Le coude (Elbow Method)

**Principe** : On teste K=2, K=3, K=4... et on mesure l'**inertie** (= a quel point les produits sont proches du centre de leur groupe).

> **Analogie** : Imaginez que vous placez des drapeaux dans un champ pour indiquer le point de rassemblement de chaque equipe. L'inertie, c'est la distance totale que tous les joueurs doivent parcourir pour rejoindre leur drapeau. Plus on met de drapeaux, plus cette distance diminue.

**Comment lire le graphique ?** L'inertie baisse toujours quand on ajoute des clusters. On cherche le **coude** : le point ou ajouter un cluster supplementaire n'ameliore plus beaucoup l'inertie. C'est comme ajouter des drapeaux qui n'aident plus personne.

### Methode 2 : Le score silhouette

**Principe** : Pour chaque produit, on mesure s'il est **bien place** dans son cluster.

> **Analogie du restaurant** : Imaginez un client assis a une table. Le score silhouette mesure : "Est-il plus proche des gens de SA table que des gens de la table la plus proche ?"
> - Score proche de **+1** : le client est bien a sa table (cluster coherent)
> - Score proche de **0** : le client pourrait etre a l'une ou l'autre table (frontiere floue)
> - Score proche de **-1** : le client serait mieux a une autre table (mauvais clustering)

**Comment lire le graphique ?** On prend le K qui donne le **score silhouette le plus eleve**.

In [ ]:
# ============================================================
# Cellule 4 : Methode du coude + Score silhouette
# ============================================================

# Plage de K a tester
K_range = range(2, 11)
inertias = []
silhouettes = []

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouettes.append(silhouette_score(X_scaled, labels))

# Graphiques
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Methode du coude
axes[0].plot(K_range, inertias, "bo-", linewidth=2, markersize=8)
axes[0].set_xlabel("Nombre de clusters (K)")
axes[0].set_ylabel("Inertie")
axes[0].set_title("Methode du coude (Elbow Method)")
axes[0].grid(True, alpha=0.3)

# Score silhouette
axes[1].plot(K_range, silhouettes, "rs-", linewidth=2, markersize=8)
axes[1].set_xlabel("Nombre de clusters (K)")
axes[1].set_ylabel("Score silhouette")
axes[1].set_title("Score silhouette en fonction de K")
axes[1].grid(True, alpha=0.3)

# Identification du meilleur K par silhouette
meilleur_k = list(K_range)[np.argmax(silhouettes)]
axes[1].axvline(x=meilleur_k, color="green", linestyle="--", linewidth=2,
                label=f"K optimal = {meilleur_k}")
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\n--- Resultats ---")
for k, s in zip(K_range, silhouettes):
    marqueur = " <-- OPTIMAL" if k == meilleur_k else ""
    print(f"  K={k} : Silhouette = {s:.4f}{marqueur}")

print(f"\nNombre optimal de clusters retenu : K = {meilleur_k}")

## K-Means Clustering

Nous allons appliquer K-Means avec le nombre optimal de clusters identifie.

In [ ]:
# ============================================================
# Cellule 5 : K-Means avec le K optimal
# ============================================================

# Application de K-Means
kmeans_final = KMeans(n_clusters=meilleur_k, random_state=42, n_init=10)
df["cluster_kmeans"] = kmeans_final.fit_predict(X_scaled)

print(f"K-Means avec K={meilleur_k} clusters")
print(f"Inertie finale : {kmeans_final.inertia_:.2f}")
print(f"Score silhouette : {silhouette_score(X_scaled, df['cluster_kmeans']):.4f}")

# Nombre de produits par cluster
print("\n--- Repartition des produits par cluster ---")
for c in sorted(df["cluster_kmeans"].unique()):
    n = (df["cluster_kmeans"] == c).sum()
    print(f"  Cluster {c} : {n} produits ({n/len(df)*100:.1f}%)")

# Diagramme de la silhouette par echantillon
fig, ax = plt.subplots(figsize=(8, 6))

sample_silhouette_values = silhouette_samples(X_scaled, df["cluster_kmeans"])
y_lower = 10

couleurs = plt.cm.Set2(np.linspace(0, 1, meilleur_k))

for i in range(meilleur_k):
    # Valeurs silhouette des echantillons du cluster i
    ith_cluster_values = sample_silhouette_values[df["cluster_kmeans"] == i]
    ith_cluster_values.sort()

    size_cluster_i = ith_cluster_values.shape[0]
    y_upper = y_lower + size_cluster_i

    ax.fill_betweenx(
        np.arange(y_lower, y_upper),
        0,
        ith_cluster_values,
        facecolor=couleurs[i],
        edgecolor=couleurs[i],
        alpha=0.7,
    )
    ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i), fontweight="bold")
    y_lower = y_upper + 10

ax.set_title("Diagramme silhouette par echantillon")
ax.set_xlabel("Score silhouette")
ax.set_ylabel("Echantillons par cluster")
ax.axvline(x=silhouette_score(X_scaled, df["cluster_kmeans"]), color="red", linestyle="--",
           label=f"Moyenne = {silhouette_score(X_scaled, df['cluster_kmeans']):.3f}")
ax.legend()
plt.tight_layout()
plt.show()

## DBSCAN : une approche basee sur la densite

### K-Means vs DBSCAN : deux philosophies

**K-Means** dit : "Je veux exactement K groupes, et chaque produit appartient a un groupe."

**DBSCAN** dit : "Je vais trouver les zones denses naturellement, et les produits isoles sont du bruit."

> **Analogie de la soiree** : K-Means, c'est un animateur qui dit "Formez 4 groupes !". DBSCAN, c'est observer naturellement ou les gens se regroupent — et accepter que certains restent seuls.

### Comment ca marche ?

DBSCAN utilise 2 parametres :
- **eps** (epsilon) : "Quelle distance maximale entre deux points pour qu'ils soient consideres voisins ?"
  - Petit eps → beaucoup de petits clusters + beaucoup de bruit
  - Grand eps → peu de gros clusters + peu de bruit

- **min_samples** : "Combien de voisins minimum pour former un cluster ?"
  - Petit min_samples → detecte des micro-groupes
  - Grand min_samples → n'accepte que les gros groupes denses

### Les avantages de DBSCAN

| | K-Means | DBSCAN |
|---|---------|--------|
| Nombre de clusters | A definir a l'avance | Detecte automatiquement |
| Forme des clusters | Spheriques uniquement | N'importe quelle forme |
| Outliers | Force dans un cluster | Les identifie comme "bruit" (label = -1) |
| Sensibilite | A l'initialisation | Aux parametres eps et min_samples |

In [ ]:
# ============================================================
# Cellule 6 : DBSCAN
# ============================================================

# Test de plusieurs valeurs d'eps
print("--- Test de differentes valeurs d'eps ---")
for eps in [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]:
    dbscan = DBSCAN(eps=eps, min_samples=3)
    labels = dbscan.fit_predict(X_scaled)
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_bruit = (labels == -1).sum()
    sil = silhouette_score(X_scaled, labels) if n_clusters >= 2 else -1
    print(f"  eps={eps:.1f} : {n_clusters} clusters, {n_bruit} points de bruit, silhouette={sil:.4f}")

# Application de DBSCAN avec les parametres retenus
eps_choisi = 2.0
min_samples_choisi = 3

dbscan_final = DBSCAN(eps=eps_choisi, min_samples=min_samples_choisi)
df["cluster_dbscan"] = dbscan_final.fit_predict(X_scaled)

n_clusters_dbscan = len(set(df["cluster_dbscan"])) - (1 if -1 in df["cluster_dbscan"].values else 0)
n_bruit = (df["cluster_dbscan"] == -1).sum()

print(f"\n--- DBSCAN (eps={eps_choisi}, min_samples={min_samples_choisi}) ---")
print(f"  Nombre de clusters : {n_clusters_dbscan}")
print(f"  Points de bruit    : {n_bruit} ({n_bruit/len(df)*100:.1f}%)")

if n_clusters_dbscan >= 2:
    # Score silhouette (sans les points de bruit)
    masque = df["cluster_dbscan"] != -1
    if masque.sum() > n_clusters_dbscan:
        sil_dbscan = silhouette_score(X_scaled[masque], df.loc[masque, "cluster_dbscan"])
        print(f"  Score silhouette   : {sil_dbscan:.4f} (hors bruit)")

print("\n--- Repartition des produits ---")
for c in sorted(df["cluster_dbscan"].unique()):
    n = (df["cluster_dbscan"] == c).sum()
    label = f"Cluster {c}" if c != -1 else "Bruit (outliers)"
    print(f"  {label} : {n} produits")

## Visualisation des clusters en 2D (PCA)

### Le probleme : comment voir en 6 dimensions ?

Nos produits sont decrits par 6 variables (prix, ventes, notes...). C'est comme avoir 6 axes sur un graphique — impossible a dessiner !

### La solution : la PCA (Analyse en Composantes Principales)

La PCA **ecrase** les 6 dimensions en 2, en conservant un maximum d'information.

> **Analogie de la photo** : Imaginez une statue en 3D. Quand vous prenez une photo, vous ecrasez les 3 dimensions en 2D. Selon l'angle, la photo capture plus ou moins bien la forme de la statue. La PCA choisit automatiquement le **meilleur angle** pour conserver le maximum d'information.

**La "variance expliquee"** vous dit quelle proportion de l'information est conservee. Par exemple, "65% de variance expliquee" signifie que la projection 2D capture 65% des differences entre les produits. C'est une bonne approximation mais pas parfaite.

> **A retenir** : Les clusters que vous voyez sur le graphique 2D sont une **simplification**. Deux points qui semblent proches en 2D peuvent etre plus eloignes dans l'espace reel a 6 dimensions.

In [ ]:
# ============================================================
# Cellule 7 : Visualisation PCA 2D
# ============================================================

# Reduction de dimension avec PCA
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

print(f"Variance expliquee par les 2 composantes : {pca.explained_variance_ratio_.sum()*100:.1f}%")
print(f"  PC1 : {pca.explained_variance_ratio_[0]*100:.1f}%")
print(f"  PC2 : {pca.explained_variance_ratio_[1]*100:.1f}%")

# Visualisation K-Means vs DBSCAN
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# K-Means
scatter1 = axes[0].scatter(
    X_pca[:, 0], X_pca[:, 1],
    c=df["cluster_kmeans"],
    cmap="Set2",
    alpha=0.7,
    edgecolors="k",
    linewidth=0.5,
    s=60,
)

# Centroides K-Means en PCA
centroides_pca = pca.transform(kmeans_final.cluster_centers_)
axes[0].scatter(
    centroides_pca[:, 0], centroides_pca[:, 1],
    c="red", marker="X", s=200, edgecolors="black", linewidth=2,
    label="Centroides",
)

axes[0].set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
axes[0].set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
axes[0].set_title(f"K-Means (K={meilleur_k})")
axes[0].legend()

# DBSCAN
# Les points de bruit (-1) sont en gris
couleurs_dbscan = df["cluster_dbscan"].copy()
masque_bruit = couleurs_dbscan == -1

# Points clusters
scatter2 = axes[1].scatter(
    X_pca[~masque_bruit, 0], X_pca[~masque_bruit, 1],
    c=couleurs_dbscan[~masque_bruit],
    cmap="Set2",
    alpha=0.7,
    edgecolors="k",
    linewidth=0.5,
    s=60,
    label="Clusters",
)

# Points de bruit
if masque_bruit.any():
    axes[1].scatter(
        X_pca[masque_bruit, 0], X_pca[masque_bruit, 1],
        c="gray", marker="x", s=50, alpha=0.5,
        label=f"Bruit ({masque_bruit.sum()} pts)",
    )

axes[1].set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
axes[1].set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
axes[1].set_title(f"DBSCAN (eps={eps_choisi}, min_samples={min_samples_choisi})")
axes[1].legend()

plt.tight_layout()
plt.show()

# Contribution des features aux composantes principales
print("\n--- Contribution des features aux composantes principales ---")
pca_contrib = pd.DataFrame(
    pca.components_.T,
    columns=["PC1", "PC2"],
    index=features_avec_cat,
).round(3)
print(pca_contrib)

## Profilage des clusters

Pour interpreter les clusters, nous allons calculer les statistiques moyennes de chaque cluster
et identifier les caracteristiques dominantes de chaque groupe.

In [ ]:
# ============================================================
# Cellule 8 : Profilage des clusters
# ============================================================

# Statistiques moyennes par cluster K-Means
variables_profil = ["prix", "nb_ventes_mensuel", "note_moyenne", "nb_avis", "stock"]

print("=" * 70)
print("  PROFILAGE DES CLUSTERS (K-Means)")
print("=" * 70)

profil = df.groupby("cluster_kmeans")[variables_profil].mean().round(2)
profil["nb_produits"] = df.groupby("cluster_kmeans").size()
print(profil)

# Repartition des categories par cluster
print("\n--- Repartition des categories par cluster ---")
cat_par_cluster = pd.crosstab(df["cluster_kmeans"], df["categorie"], margins=True)
print(cat_par_cluster)

# Visualisation du profil moyen de chaque cluster (radar chart simplifie)
fig, axes = plt.subplots(1, meilleur_k, figsize=(5 * meilleur_k, 5))
if meilleur_k == 1:
    axes = [axes]

# Normalisation des moyennes pour comparaison visuelle
profil_norm = profil[variables_profil].copy()
for col in variables_profil:
    col_min = profil_norm[col].min()
    col_max = profil_norm[col].max()
    if col_max > col_min:
        profil_norm[col] = (profil_norm[col] - col_min) / (col_max - col_min)
    else:
        profil_norm[col] = 0.5

couleurs_cluster = plt.cm.Set2(np.linspace(0, 1, meilleur_k))

for i in range(meilleur_k):
    valeurs = profil_norm.loc[i].values
    axes[i].barh(variables_profil, valeurs, color=couleurs_cluster[i], edgecolor="black")
    axes[i].set_xlim(0, 1.1)
    axes[i].set_title(f"Cluster {i} ({profil.loc[i, 'nb_produits']} produits)", fontweight="bold")
    axes[i].set_xlabel("Valeur normalisee")

plt.tight_layout()
plt.show()

# Heatmap des profils
fig, ax = plt.subplots(figsize=(10, 5))
sns.heatmap(
    profil_norm,
    annot=profil[variables_profil].round(1).values,
    fmt="",
    cmap="YlOrRd",
    ax=ax,
    linewidths=1,
    xticklabels=variables_profil,
    yticklabels=[f"Cluster {i}" for i in range(meilleur_k)],
)
ax.set_title("Profil moyen des clusters (valeurs reelles annotees)")
plt.tight_layout()
plt.show()

# Exemples de produits par cluster
print("\n--- Exemples de produits par cluster ---")
for c in sorted(df["cluster_kmeans"].unique()):
    print(f"\n  Cluster {c} :")
    exemples = df[df["cluster_kmeans"] == c][["nom_produit", "categorie", "prix", "nb_ventes_mensuel"]].head(5)
    for _, row in exemples.iterrows():
        print(f"    - {row['nom_produit']} ({row['categorie']}) | {row['prix']}e | {row['nb_ventes_mensuel']} ventes/mois")

## Conclusion et insights business

### Recapitulatif methodologique

| Methode | Avantages | Inconvenients |
|---------|-----------|---------------|
| **K-Means** | Simple, rapide, interpretable | Nombre de clusters a definir, clusters spheriques |
| **DBSCAN** | Detection d'outliers, formes arbitraires | Parametres eps/min_samples a regler |

### Interpretation des clusters

L'analyse des profils moyens permet d'attribuer des labels metier aux clusters.
Par exemple :

- **Produits premium** : prix eleve, peu de ventes, note elevee
- **Best-sellers** : prix moyen/bas, beaucoup de ventes, beaucoup d'avis
- **Produits de niche** : prix variable, peu de ventes, peu d'avis
- **Produits populaires abordables** : prix bas, ventes elevees, stock important

### Applications metier

- **Marketing** : adapter la communication par segment de produits
- **Pricing** : ajuster les prix selon le cluster
- **Stock** : optimiser les niveaux de stock par segment
- **Cross-selling** : recommander des produits du meme cluster

### Pour aller plus loin

- Tester d'autres algorithmes (Agglomeratif, Gaussian Mixture Models)
- Utiliser t-SNE ou UMAP pour une meilleure visualisation 2D
- Combiner le clustering avec des donnees clients pour une segmentation croisee
- Automatiser le choix des hyperparametres avec une recherche en grille

### Lexique debutant

| Terme | Definition simple |
|-------|------------------|
| **Clustering** | Regrouper des elements similaires sans categories predefinies |
| **Apprentissage non supervise** | Apprendre sans etiquettes (pas de "bonne reponse" connue) |
| **K-Means** | Algorithme qui cree K groupes en minimisant les distances |
| **DBSCAN** | Algorithme qui detecte les zones denses et identifie le bruit |
| **Inertie** | Distance totale entre chaque point et le centre de son cluster |
| **Score silhouette** | Mesure si chaque point est bien dans son cluster (-1 a +1) |
| **PCA** | Technique pour reduire le nombre de dimensions (pour visualiser) |
| **Centroide** | Le "point central" d'un cluster (la moyenne de tous ses membres) |
| **Outlier / Bruit** | Point isole qui n'appartient a aucun cluster |
| **Standardisation** | Remettre toutes les variables sur la meme echelle |

In [ ]:
# ============================================================
# Cellule 9 : Resume final
# ============================================================

print("=" * 60)
print("  RESUME FINAL - SEGMENTATION DES PRODUITS")
print("=" * 60)

print(f"\n  Nombre de produits analyses  : {len(df)}")
print(f"  Nombre de features utilisees : {len(features_avec_cat)}")
print(f"  Algorithme retenu            : K-Means")
print(f"  Nombre de clusters           : {meilleur_k}")
print(f"  Score silhouette             : {silhouette_score(X_scaled, df['cluster_kmeans']):.4f}")

print(f"\n--- Profil synthetique des clusters ---")
for c in sorted(df["cluster_kmeans"].unique()):
    sous_df = df[df["cluster_kmeans"] == c]
    prix_moy = sous_df["prix"].mean()
    ventes_moy = sous_df["nb_ventes_mensuel"].mean()
    note_moy = sous_df["note_moyenne"].mean()
    categories = sous_df["categorie"].mode().iloc[0]
    
    print(f"\n  Cluster {c} ({len(sous_df)} produits) :")
    print(f"    Prix moyen     : {prix_moy:.2f} euros")
    print(f"    Ventes/mois    : {ventes_moy:.0f}")
    print(f"    Note moyenne   : {note_moy:.2f}/5")
    print(f"    Categorie dom. : {categories}")

print("\n" + "=" * 60)